# Lab 1 - Convolutional neural networks

### Task 1 (15 points)
Select the dataset:
- [CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html)

Extra credit for the dataset below
- [ImageNet](http://www.image-net.org/) *
- [300-W](https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/)
- [IMDB-WIKI](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/) *
- [UTKFace](https://susanqq.github.io/UTKFace/)
- Any other, but in this case, please check with me. For example, something from [here](https://huggingface.co/datasets?task_categories=task_categories:image-classification) or [here](https://www.tensorflow.org/datasets/catalog/overview).

<i>*since the dataset is very large, you can take a part of it</i>

Choose an architecture:
- VGGNet
- Inception (v1-v4)
- Inception-ResNet-v2
- ResNet
- Xception
- Any other, but in this case, please coordinate with me

Solve a classification or regression problem with the selected architecture for the selected dataset. Implement a neural network from scratch, you cannot take a ready-made one.

It is highly recommended to train on GPU. If you do not have a CUDA-compatible video card, use [Google Collaboratory](https://colab.research.google.com).

In [1]:
%pip install tensorflow 
%pip install matplotlib 
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (227, 227))
    return image, label

def get_data(train_images, train_labels, test_images, test_labels):
    validation_images, validation_labels = train_images[:5000], train_labels[:5000]
    train_images, train_labels = train_images[5000:], train_labels[5000:]

    train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

    train_ds = (train_ds
                .map(process_images, num_parallel_calls=tf.data.AUTOTUNE)
                #.shuffle(buffer_size=1000)
                .batch(batch_size=32, drop_remainder=True)
                .prefetch(tf.data.AUTOTUNE))
    test_ds = (test_ds
               .map(process_images, num_parallel_calls=tf.data.AUTOTUNE)
               #.shuffle(buffer_size=1000)
               .batch(batch_size=32, drop_remainder=True)
               .prefetch(tf.data.AUTOTUNE))
    validation_ds = (validation_ds
                     .map(process_images, num_parallel_calls=tf.data.AUTOTUNE)
                     #.shuffle(buffer_size=1000)
                     .batch(batch_size=32, drop_remainder=True)
                     .prefetch(tf.data.AUTOTUNE))

    return train_ds, validation_ds, test_ds


(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
train_ds, validation_ds, test_ds = get_data(train_images, train_labels, test_images, test_labels)

In [5]:
from tensorflow.keras import layers, models

def residual_block(x, filters, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, (3, 3), strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, (3, 3), strides=1, padding="same")(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or x.shape[-1] != shortcut.shape[-1]:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding="same")(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x

def ResNet(input_size, output_size):
    inputs = layers.Input(shape=input_size)
    x = layers.Conv2D(64, (7, 7), strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding="same")(x)

    # Residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)

    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(output_size, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model


In [7]:
checkpoint_path = "best_resnet_model.keras"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

model = ResNet(input_size=(227, 227, 3), output_size=10)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=40,
    callbacks=[checkpoint_cb]
)

Epoch 1/40
  14/1406 ━━━━━━━━━━━━━━━━━━━━ 56:24 2s/step - accuracy: 0.1221 - loss: 2.4797

KeyboardInterrupt: 

using colab with gpu for fast training

In [8]:
best_model = keras.models.load_model(checkpoint_path)

test_loss, test_accuracy = best_model.evaluate(test_ds)
print(f"\nBest Model Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

312/312 ━━━━━━━━━━━━━━━━━━━━ 204s 643ms/step - accuracy: 0.8352 - loss: 1.0235

Best Model Test Accuracy: 0.8361, Test Loss: 1.0322


### Task 1 (alternative) (15 points + bonus on the exam :))

Choose something from the list:
- Solve the problem of finding objects in an image:
    - [Faces](https://www.tensorflow.org/datasets/catalog/wider_face)
    - [Traffic objects](https://www.tensorflow.org/datasets/catalog/kitti)
    - Something [from here](https://huggingface.co/datasets?task_categories=task_categories:object-detection&sort=downloads)
    - Your choice
- Semantic segmentation:
    - [MRI brain scans](https://www.kaggle.com/mateuszbuda/lgg-mri-segmentation)
    - [Traffic objects](https://www.tensorflow.org/datasets/catalog/kitti)
    - Something [from here](https://huggingface.co/datasets?task_categories=task_categories:image-segmentation&sort=downloads)
    - Your choice

### Task 2 (15 points)
Prepare your trained model for deployment:
- **(5 points)** Convert model to Tensorflow Lite or ONNX;
- **(5 points)** Setup the inference API server using any web-framework (Flask, FastAPI, etc.).
  The inference API must contain `/predict` endpoint, which accepts an image and returns model prediction;
- **(5 points)** Create Docker image for the inference API.

Prepare a report demonstrating the process of running and querying the API

#### * Convert to Tensorflow Lite format

In [9]:
# Convert to Tensorflow Lite format

model = tf.keras.models.load_model("best_resnet_model.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("resnet_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite format.")

INFO:tensorflow:Assets written to: C:\Users\0E46~1\AppData\Local\Temp\tmpio6xs8k0\assets


INFO:tensorflow:Assets written to: C:\Users\0E46~1\AppData\Local\Temp\tmpio6xs8k0\assets


Saved artifact at 'C:\Users\0E46~1\AppData\Local\Temp\tmpio6xs8k0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1943334170112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334169056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334191696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334191872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334177072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334178304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334195040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334230848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334253136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1943334254192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  19